In [57]:
import spacy
import json
import re
from typing import List
from spacy.tokenizer import Tokenizer

In [2]:
def read_json(file: str):
    with open(file) as f:
        data = json.load(f)
    
    return data

# Заголовки новин

## 1. Форматування

### 1. напишіть програму, яка форматує заголовки за вказаними правилами

1) З великої літери потрібно писати слова довжиною 4 чи більше літер. <br/>
2) З великої літери потрібно писати перше і останнє слово заголовку, незалежно від частини мови. <br/>
3) З великої літери потрібно писати іменники, займенники, дієслова, прикметники, прислівники та підрядні сполучники. <br/>
4) Якщо слово написане через дефіс, велику літеру потрібно додати для кожної частинки слова (наприклад, правильно "Self-Reflection", а не "Self-reflection"). <br/>
5) З маленької літери потрібно писати всі інші частини мови: артиклі/визначники, сурядні сполучники, прийменники, частки, вигуки. <br/>

In [230]:
class HeadlineFormatter:
    
    TITLE_POS_TAGS = ['NOUN', 'PRON', 'PROPN', 'VERB', 'ADJ', 'ADV', 'SCONJ', 'AUX']
    LOWER_POS_TAGS = ['DET', 'CCONJ', 'PREP', 'PART', 'INTJ']
    
    LOWER_EXCEPTION = ["n't", "'s"]
    
    def __init__(self):
        nlp = spacy.load("en_core_web_sm")
        self.__tokenizer = spacy.load("en_core_web_sm")
        self.__upper_pattern = re.compile("[A-Z]+")

    
    def format(self, headline: str) -> str:
        tokens = self.__tokenize(headline)
        size = len(tokens)
        
        text_with_ws_list = []

        prev_title = False
        is_prev_punct = False
        
        start_quotes = False
    
        for i in range(0, size):
            token = tokens[i]
            text_with_ws = token.text_with_ws
            
            
#             if token.pos_ == 'PUNCT' and token.text == "'":
#                 if start_quotes:
#                     start_quotes = False
#                 else:
#                     start_quotes = True
            
            if start_quotes is True:
                text_with_ws_list.append(text_with_ws)
                continue

            if token.pos_ == 'PUNCT':
                is_prev_punct = True
                text_with_ws_list.append(text_with_ws)
                continue
                
            if len(token.text) > 1 and self.__is_upper(token.text):
                text_with_ws_list.append(text_with_ws)
                continue


            
            if i == 0 or i == size -1 or self.__should_be_title(token):
                text_with_ws = self.__title_token(token)
                prev_title = True
            else:
                if prev_title and is_prev_punct:
                    text_with_ws = self.__title_token(token)
                    prev_title = True
                else:
                    prev_title = False
                    
                    if self.__should_be_lower(token):
                        text_with_ws = text_with_ws.lower()
                    

            text_with_ws_list.append(text_with_ws)
            is_prev_punct = False
                    
        return self.__untokenize(text_with_ws_list)
    
    def __should_be_title(self, token):
        if self.__is_article(token.text) is True:
            return False
        
        if token.text in self.LOWER_EXCEPTION:
            return False
        
        if token.pos_ in self.TITLE_POS_TAGS:
            return True
        
        if len(token.text) >= 4:
            return True
        
        return False
    
    def __should_be_lower(self, token):
        if self.__is_article(token.text) is True:
            return True
        
        if token.pos_ in self.LOWER_POS_TAGS:
            return True
        
        return False


    def __is_upper(self, text):
        if self.__upper_pattern.fullmatch(text) is not None:
            return True
        return False
    
    def __is_article(self, text: str):
        if text.lower() in ['a', 'an', 'the']:
            return True
        
        return False


        
                
    def __title_token(self, token)-> str:
        return token.text_with_ws.title()
    
    def __title(self, token):
        #TODO: consider "-
        return token.text_with_ws.title()
    
    
    def __tokenize(self, text):
        return self.__tokenizer(text)
    
    def __untokenize(self, text_with_ws_list):
        return ''.join(text_with_ws for text_with_ws in text_with_ws_list)
        

In [231]:
headline_formatter = HeadlineFormatter()

In [232]:
headline_formatter.format("Dicks Creek: Georgia's Go-to Trout Water")

"Dicks Creek: Georgia's Go-To Trout Water"

In [233]:
headline_formatter.format("Friday Fun: Project Runway's Kayne at SWS, Manuel dances for charity, Laura Bell Bundy sings at PLAY")

"Friday Fun: Project Runway's Kayne at SWS, Manuel Dances for Charity, Laura Bell Bundy Sings at PLAY"

### 2. перевірте якість роботи програми на валідаційній вибірці

In [260]:
def calculate_accuracy(expected_headlines: List[str], actual_headlines: List[str], headlines):
    
    assert len(expected_headlines) == len(actual_headlines)
    tp = 0
    
    for i in range(0, len(expected_headlines)):
        if expected_headlines[i] == actual_headlines[i]:
            tp += 1
    accuracy = tp / len(expected_headlines)
    
    return accuracy

In [261]:
val_file = "../../../tasks/02-structural-linguistics/data/headlines-test-set.json"

In [262]:
val_data = read_json(val_file)

In [263]:
expected_headlines = [item[1] for item in val_data]

In [264]:
headlines = [item[0] for item in val_data]

In [265]:
headline_formatter = HeadlineFormatter()

In [266]:
formatted_headlines = [headline_formatter.format(headline) for headline in headlines]

In [267]:
accuracy = calculate_accuracy(expected_headlines=expected_headlines, 
                              actual_headlines=formatted_headlines, 
                              headlines=headlines)

#### Accuracy

In [268]:
print("Accuracy: ", accuracy)

Accuracy:  0.69


### 3. проженіть вашу програму на корпусі заголовків з The Examiner і вирахуйте, скільки заголовків там відформатовано за правилами (скільки заголовків залишились незмінними після форматування)

In [254]:
def read_file(file: str):
    lines = []
    with open(file) as file_in:        
        for line in file_in:
            lines.append(line)
            
    return lines

In [255]:
examiner_file = "../../../tasks/02-structural-linguistics/data/examiner-headlines.txt"

In [256]:
examiner_lines = read_file(examiner_file)

In [258]:
headline_formatter = HeadlineFormatter()

In [259]:
num_changed = 0
for headline in examiner_lines:
    headline_formatted = headline_formatter.format(headline)
    if headline_formatted != headline:
        num_changed += 1
        
size = len(examiner_lines)
print(f"There were formatted {num_changed} out of {size}")
print(f"There were not  formatted {size - num_changed} out of {size}")

There were formatted 4412 out of 5000
There were not  formatted 588 out of 5000


## 2. Вірусні новини